# Milestone 1: Groups, Data, Website, and Extraction, Transform, and Load (ETL)

Terrorism causes fear and terror, as it is aptly named, across the world. People commit acts of terror for political, religious, and personal reasons along with many more. In addition, these are committed in multiple ways — e.g. bombing, kidnapping, assault, etc. — with a variety of weapons such as IEDs, assault rifles, poison, etc. Terrorism has often been used as a weapon of fear, not by the terrorists themselves, but by the state that has just been attacked to further their goals by convincing their populace their goals must be achieved to prevent more terror. Through the Global Terrorism Database (GTD), I hope to robustly analyze data of acts of terror from 1970 to 2019. 

How does one classify an attack as that of a terrorist? 
In what countries are certain acts of terror most committed?
Can we predict what country or region will be attacked given their method and target of terror?

Fortunately, the GTD appears to have the answer to all of the above and more, but the analysis can be potentially taken further with the inclusion of the CIA world factbook (CWF). Regarding the terrorists themselves, people do not simply one day wake up and decide to commit an act of terror. There are many outside factors that can affect such a drastic decision. People from countries with low unemployment, low GDP, and low education may have a higher chance of committing acts of terror. The world factbook contains information on GDP, natural resources, unemployment, and more that can be cross-referenced with the GDT enabling access to more potential predictors (and overall data as a whole) leading to a far more thorough analysis. More questions can be asked such as:

Do countries with high natural resources get terrorized more than those without them, or is it only those with oil and natural gas? 
How does military expending affect the frequency of terror? Or does large military spending offer more or less protection depending on GDP?
Can we use all of the above along with the target type, attack type, and nationality to predict which country has a high chance of being attacked? 


## Reading in Data

In [892]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [869]:
%matplotlib inline
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.pyplot import cm
from matplotlib.patches import Patch

pd.set_option('display.max_columns', None)


Read in the GDT data

In [901]:
df = pd.read_excel('./data/globalterrorismdb_0221dist.xlsx')
df_1993 = pd.read_excel('./data/gtd1993_0221dist.xlsx')

The 1993 data was left out, so it must be added to the data set and placed into the right location, between 1992 and 1994.

In [902]:
gtd_df = pd.concat([df_1993, df])
gtd_df = gtd_df.sort_values(by=['eventid'])
gtd_df = gtd_df.reset_index()
gtd_df.drop(['index'],axis=1)

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,targtype3,targtype3_txt,targsubtype3,targsubtype3_txt,corp3,target3,natlty3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaT,58,Dominican Republic,2,Central America & Caribbean,National,Santo Domingo,18.456792,-69.951164,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1.0,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,68.0,Named Civilian,NaN,Julio Guzman,58.0,Dominican Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MANO-D,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaT,130,Mexico,1,North America,Federal,Mexico city,19.371887,-99.086624,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1.0,0,6,Hostage Taking (Kidnapping),NaN,NaN,NaN,NaN,7,Government (Diplomatic),45.0,"Diplomatic Personnel (outside of embassy, cons...",Belgian Ambassador Daughter,"Nadine Chaval, daughter",21.0,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23rd of September Communist League,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,NaN,Mexico,1.0,800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaT,160,Philippines,5,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,4.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1.0,0,1,Assassination,NaN,NaN,NaN,NaN,10,Journalists & Media,54.0,Radio Journalist/Staff/Facility,Voice of America,Employee,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaT,78,Greece,8,Western Europe,Attica,Athens,37.997490,23.762728,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1.0,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,7,Government (Diplomatic),46.0,Embassy/Consulate,NaN,U.S. Embassy,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,16.0,Unknown Explosive Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosive,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Na

Create datetime objects and add to new column 'date' to enable easy date access.

In [ ]:
gtd_df['date'] = 0

for i, (y, m, d) in enumerate(zip(gtd_df['iyear'], gtd_df['imonth'], gtd_df['iday'])):
    tmp = []
    tmp.append(i)
    if m == 0:
        gtd_df.loc[i,'date'] = pd.to_datetime(y, format='%Y')
    else:
        if d == 0:
            gtd_df.loc[i, 'date'] = pd.to_datetime(str(y)+str(m), format='%Y%m')
        else:
            gtd_df.loc[i, 'date'] = pd.to_datetime(str(y)+str(m)+str(d), format='%Y%m%d',errors='coerce')

Drop columns that do not appear to be of any use. 

Rename columns with multiple dropped types.

Replace data indicating nothing with NaNs.

In [ ]:
drop_cols = ['extended','resolution','vicinity','latitude','longitude','gname2','gsubname2','gname3','gsubname3',
             'guncertain2','guncertain3','claimmode','claimmode_txt','claim2','claimmode2','claimmode2_txt',
             'claim3','claimmode3','claimmode3_txt','compclaim','nhours','ndays','divert','ransomnote','addnotes',
             'scite1','scite2','scite3','dbsource','imonth','iday','approxdate']
# Potential drops: specificity, multiple, related
gtd_df.drop(columns=drop_cols)
gtd_df = gtd_df.rename(columns={'attacktype1':'attacktype','attacktype1_txt':'attacktype_txt','targtype1':'targtype',
                                'targtype1_txt':'targtype_txt', 'natlty1':'natlty','natlty1_txt':'natlty_txt'})
gtd_df = gtd_df.replace(-99, np.nan)
gtd_df = gtd_df.replace(-9, np.nan)

# Milestone 2: Additional Extraction, Transform, and Load (ETL) + Exploratory Data Analysis (EDA)

A basic analysis of the regions with the most acts of terror show the Middle East & Northern Africa along with South Asia in a clear lead, most likely due to greater political instability within the regions.

In [ ]:
fig, ax = plt.subplots(1,1)
region_cts = gtd_df['region_txt'].value_counts()
fig = region_cts.plot.bar(figsize=(20,10))

Create dataframes per region along with a list of region names for ease of use later.

In [ ]:
region_dfs = [gtd_df[gtd_df['region_txt'] == 'Western Europe'],
              gtd_df[gtd_df['region_txt'] == 'North America'],
              gtd_df[gtd_df['region_txt'] == 'Middle East & North Africa'],
              gtd_df[gtd_df['region_txt'] == 'South America'],
              gtd_df[gtd_df['region_txt'] == 'Southeast Asia'],
              gtd_df[gtd_df['region_txt'] == 'Sub-Saharan Africa'],
              gtd_df[gtd_df['region_txt'] == 'South Asia'],
              gtd_df[gtd_df['region_txt'] == 'Central America & Caribbean'],
              gtd_df[gtd_df['region_txt'] == 'Eastern Europe'],
              gtd_df[gtd_df['region_txt'] == 'East Asia'],
              gtd_df[gtd_df['region_txt'] == 'Australasia & Oceania'],
              gtd_df[gtd_df['region_txt'] == 'Central Asia']
             ]
region_names = ['Western Europe', 'North America', 'Middle East & North Africa', 'South America', 'Southeast Asia', 
                'Sub-Saharan Africa', 'South Asia', 'Central America & Caribbean', 'Eastern Europe', 'East Asia', 
                'Australasia & Oceania', 'Central Asia']

Below illustrates the proportions in two separate graphs of terrorist attacks and targets along with their relative size with respect to other regions. 

In [ ]:
attacktype_cts = gtd_df['attacktype_txt'].value_counts()
targtype_cts = gtd_df['targtype_txt'].value_counts()

targ_region_cts = pd.crosstab(gtd_df['region_txt'], gtd_df['targtype_txt'], normalize=True)
atk_region_cts = pd.crosstab(gtd_df['region_txt'], gtd_df['attacktype_txt'], normalize=True)
fig1 = atk_region_cts.plot.bar(stacked=True, figsize=(20,10), 
                               title='Types of Terrorist Attacks & Proportions vs. Regions')
fig2 = targ_region_cts.plot.bar(stacked=True, figsize=(20,10), 
                                title= 'Types of Terrorist Attacks & Proportions vs. Regions')


Next, we look at the proportions of the types of attacks on specific targets. It makes sense that abortion related acts 
of terror would involve a facility/infrastructure attack, as well as airports & aircraft having the highest relative proportion
of hijacking.

In [ ]:
# targtype given attack type
fig, ax = plt.subplots(1,1)
targ_region_cts = pd.crosstab(gtd_df['targtype_txt'], gtd_df['attacktype_txt'])
targ_cts = targ_region_cts.sum(axis=1)
attk_cts = targ_region_cts.sum(axis=0)
region_given_targ = targ_region_cts.divide(targ_cts, axis=0)
fig = region_given_targ.plot.bar(ax=ax, stacked=True, figsize=(15,8), title='Proportions of Attack Types on Targets')
ax.set_xlabel('Target Type')
ax.set_ylabel('% Occurrence')
ax.legend(bbox_to_anchor=(1, 1))

Interestingly enough, all terrorist attacks appear to have a staggering success rate. Central America & the Carribean have an incredibly low failure rate

In [ ]:
fig, ax = plt.subplots(4, 3)
color_dict = {0.0:'crimson', 1.0:'forestgreen'}
labels = ['Success', 'Failure']
i = 0
fig.suptitle('Success vs. Failure of Terrorist Attacks by Region', fontsize=20)
for r in range(4):
    for c in range(3):
        ax[r,c] = (pd.crosstab(region_dfs[i]['region_txt'], 
                               region_dfs[i]['success'], 
                               normalize=True)).plot.bar(ax=ax[r,c], legend=False,figsize=(20,10), color=color_dict, rot=0)
        ax[r,c].set_xlabel('')
        i+=1
ax[0, 2].legend(['Failure','Success'], bbox_to_anchor=(1.21,1))
fig.tight_layout()


In [ ]:
fig, ax = plt.subplots(4, 3)
fig.suptitle('Frequency of Terrorist Attacks Over Time by Region', fontsize=20, sharey=True)
i = 0
for r in range(4):
    for c in range(3):
        region_dfs[i].groupby('iyear').crit1.count().plot(ax=ax[r,c], figsize=(20,10), title=region_names[i])
        i += 1
        ax[r,c].set_xlabel('Year')
        ax[r,c].set_ylabel('# of Occurrences')


fig.tight_layout()

In [ ]:
tmp_df = gtd_df[['region_txt', 'targtype_txt']].copy()
terr_dummy = pd.get_dummies(tmp_df)
terr_dummy.drop(columns=['targtype_txt_Unknown'])
terr_corr = terr_dummy.corr()
terr_corr.drop(terr_corr.iloc[:, 12:], inplace = True, axis = 1)
terr_corr.drop(terr_corr.iloc[:, :12], inplace = True, axis = 0)

terr_corr.columns = terr_corr.columns.str.lstrip('region_txt_')
terr_corr = terr_corr.reset_index()
terr_corr['index'] = terr_corr['index'].str.lstrip('targtype_txt_')

In [ ]:
fig, axs = plt.subplots(4, 3, figsize=(20,10), sharey=True)
fig.suptitle('Correlation Between Region & Attack Target', fontsize=20)
i = 0
color = cm.rainbow(np.linspace(0, 1, 22))
for t, ax in enumerate(axs.ravel()):
    x = terr_corr['index']
    height = terr_corr[region_names[i]]
    ax.bar(x, height, color=color)
    ax.set_title(region_names[i])
    ax.get_xaxis().set_visible(False)
    i+=1

custom_leg = []
for i, n in enumerate(terr_corr['index']):
    custom_leg.append(Patch(facecolor=color[i], label=n))    
fig.set_figheight(15)
fig.set_figwidth(15)    

handles = terr_corr['index']
axs[1,2].legend(custom_leg, handles, bbox_to_anchor=(1,1))
fig.subplots_adjust(top=0.93)